In [2]:
from transformers import MimiModel, AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("kyutai/mimi")
model = MimiModel.from_pretrained("kyutai/mimi")
model = model.to("cuda")

/home/ritsuko/ai/audio/dual-ar/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ritsuko/ai/audio/dual-ar/.venv/lib/python3.9/site-packages/transformers/models/mimi/modeling_mimi.py:163: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [3]:
import os

def run_llama_generate(
    text="Can you generate five simple sentences for my child to practice speaking",
    temp=0.1,
    checkpoint_path="../dual-ar/checkpoints/smoltts_scratch/",
    working_dir="../../fish-speech.rs"  # Replace with your desired working directory
):
    # Store current working directory
    original_dir = os.getcwd()
    
    try:
        # Change to desired working directory
        os.chdir(working_dir)
        
        # Construct the command
        cmd = f'cargo run --release --features cuda --bin llama_generate -- '\
              f'--text "{text}" '\
              f'--checkpoint {checkpoint_path} '\
              f'--temp {temp}'
        
        # Execute command
        return os.system(cmd)
        
    finally:
        # Always return to original directory
        os.chdir(original_dir)

# Example usage:
# run_llama_generate(
#     text="Write a short story about a cat",
#     temp=0.2,
#     working_dir="/path/to/your/project"
# )

In [13]:
import numpy as np
import torch
from IPython.display import Audio, display

run_llama_generate(
    text="Here's how Bob talks, here's what language is, now speak like Bob saying this new thing",
    temp=0.05
)
# Load and process the data
test_arr = np.load("../../fish-speech.rs/out.npy")
test_input = torch.from_numpy(test_arr[:,:200]).to("cuda").to(torch.long).unsqueeze(0)

# Generate audio
out_pcm = model.decode(test_input)

# Convert to CPU and get numpy array for playback
audio_data = out_pcm.audio_values[0].detach().to("cpu").numpy()

# Create and display audio widget
# Note: sample_rate=24000 matches your original save command
display(Audio(audio_data, rate=24000, autoplay=False))

   --> fish_speech_core/lib/models/text2semantic/mod.rs:275:13
    |
275 |         let kv_seqlen = key_states.dim(2)?;
    |             ^^^^^^^^^ help: if this is intentional, prefix it with an underscore: `_kv_seqlen`
    |
    = note: `#[warn(unused_variables)]` on by default

    Finished `release` profile [optimized] target(s) in 0.13s
     Running `target/release/llama_generate --text 'Here'\''s how Bob talks, here'\''s what language is, now speak like Bob saying this new thing' --checkpoint ../dual-ar/checkpoints/smoltts_scratch/ --temp 0.05`


Model loaded to Cuda(CudaDevice(DeviceId(1)))
Text: "Here's how Bob talks, here's what language is, now speak like Bob saying this new thing"
Speaker conditioning size: [9, 10]
Loaded prompt with shape [9, 38]
Input tokens:
[1, 9690, 198, 15024, 494, 578, 260, 2711, 1694, 2, 1, 4093, 198, 4590, 506, 638, 8363, 11748, 28, 1535, 506, 732, 1789, 314, 28, 1209, 3287, 702, 8363, 4976, 451, 725, 2775, 2, 1, 520, 9531, 198]
Input prompt:
<|im_start|>system
Speak out the provided text<|im_end|><|im_start|>user
Here's how Bob talks, here's what language is, now speak like Bob saying this new thing<|im_end|><|im_start|>assistant

135.54ms prompt processing: 38 tokens (38 new, 0 cached, 280.35 tokens/s)
128 tokens generated in 0.816s (156.86 tokens/s, 6.425ms / token, RTF: 7.284)


In [3]:
from datasets import load_from_disk

dataset = load_from_disk("./tokenized_libritts_bijection")
dataset["train"][0:3]

{'normalized_text': ['[The moon] I gazed with a kind of wonder.',
  '[The moon] I gazed with a kind of wonder.',
  'I gradually saw plainly the clear stream that supplied me with drink and the trees that shaded me with their foliage.'],
 'speaker_id': ['730', '730', '730'],
 'id': ['730_358_000003_000002',
  '730_358_000003_000002',
  '730_358_000004_000001'],
 'tokens': [tensor([[    1,  9690,   198, 15024,   494,   578,   260,  2711,  1694,     2,
             198,     1,  4093,   198,    75,   504,  6783,    77,   339,   310,
           17413,   351,   253,  1942,   282,  4558,    30,     2,   198,     1,
             520,  9531,   198, 50201, 50254, 50838, 50410, 50410, 50841, 50680,
           51139, 50130, 49464, 51191, 49905, 50121, 49750, 50236, 50420, 49773,
           50909, 49712, 50886, 50679, 50269, 49774, 49780, 49662, 49775, 49775,
           50070, 49841, 50149, 50221, 51093, 49446, 49926, 49670, 51139, 49921],
          [    0,     0,     0,     0,     0,     0,     0,